# Statistics

### Comparison

In [2]:
from data import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

np.random.seed(2)
torch.manual_seed(2)


# Define the linear regression model
class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)
    
# Convert the data to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True)

# Define the lambda value
lambda_val = 10**(-8)

# List of important mse values
mse_values = []
y_true_linreg = np.array([])
y_hat_lingreg = np.array([])

for train_idx, test_idx in kf.split(X_tensor):
    x_train, x_test = X_tensor[train_idx], X_tensor[test_idx]
    y_train, y_test = y_tensor[train_idx], y_tensor[test_idx]

    model = LinearRegression(input_size=X_tensor.shape[1], output_size=1)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=lambda_val)

    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        outputs = model(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        y_pred = model(x_test)
        y_hat_lingreg = np.concatenate((y_hat_lingreg,y_pred.numpy().flatten()))
        y_true_linreg = np.concatenate((y_true_linreg,y_test.numpy().flatten()))
        
        mse = criterion(y_pred, y_test)
        mse_values.append(mse.item())
np.mean(mse_values)    

0.5176677137613297

In [3]:
error_linreg = linreg_error = (y_hat_lingreg - y_true_linreg)**2

### ANN

In [4]:
from data import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

np.random.seed(2)
torch.manual_seed(2)


class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_units):
        super(ANNModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_units)
        self.layer2 = nn.Linear(hidden_units, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x
    
# Convert the data to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True)

# Define the lambda value
hidden_units = 9
# List of important mse values
mse_values = []
y_true_ann = np.array([])
y_hat_ann = np.array([])

for train_idx, test_idx in kf.split(X_tensor):
    x_train, x_test = X_tensor[train_idx], X_tensor[test_idx]
    y_train, y_test = y_tensor[train_idx], y_tensor[test_idx]

    model = ANNModel(input_dim=X.shape[1], hidden_units=hidden_units)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)  # No regularization

    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        outputs = model(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        y_pred = model(x_test)
        y_hat_ann = np.concatenate((y_hat_ann,y_pred.numpy().flatten()))
        y_true_ann = np.concatenate((y_true_ann,y_test.numpy().flatten()))
        
        mse = criterion(y_pred, y_test)
        mse_values.append(mse.item())
np.mean(mse_values)

0.6677358567714691

In [7]:
error_ann = (y_hat_ann - y_true_ann)**2
error_linreg.shape, error_ann.shape


((442,), (442,))

TtestResult(statistic=-5.6919346119432515, pvalue=2.2942231604709003e-08, df=441)

### Baseline

In [23]:
from data import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

np.random.seed(2)
torch.manual_seed(2)
    
# Convert the data to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True)


# List of important mse values
mse_values = []
y_true_base = np.array([])
y_hat_base = np.array([])

for train_idx, test_idx in kf.split(X_tensor):
    x_train, x_test = X_tensor[train_idx], X_tensor[test_idx]
    y_train, y_test = y_tensor[train_idx], y_tensor[test_idx]

    y_mean = y_train.mean()

    y_pred = y_mean.repeat(y_test.shape[0], 1)

    y_hat_base = np.concatenate((y_hat_base,y_pred.numpy().flatten()))
    y_true_base = np.concatenate((y_true_base,y_test.numpy().flatten()))  
    mse = criterion(y_pred, y_test)
    mse_values.append(mse.item())

error_base = (y_hat_base - y_true_base)**2
np.mean(mse_values)    

1.000981366634369

In [25]:
from scipy import stats
lin_vs_ann = stats.ttest_rel(error_linreg, error_ann)
lin_vs_base = stats.ttest_rel(error_linreg, error_base)
ann_vs_base = stats.ttest_rel(error_ann, error_base)

lin_vs_ann, lin_vs_base, ann_vs_base


(TtestResult(statistic=-5.6919346119432515, pvalue=2.2942231604709003e-08, df=441),
 TtestResult(statistic=-10.325542601340365, pvalue=1.568240365173687e-22, df=441),
 TtestResult(statistic=-9.475504026424664, pvalue=1.6396805799028226e-19, df=441))

In [40]:
import numpy as np
from scipy import stats

def calculate_confidence_interval(error1, error2):
    # Calculate the difference and its mean and standard error
    difference = error1 - error2
    mean_diff = np.mean(difference)
    std_err_diff = np.std(difference, ddof=1) / np.sqrt(442)
    df = 441

    # Calculate the confidence interval
    confidence_interval = stats.t.interval(0.95, df, loc=mean_diff, scale=std_err_diff)

    return confidence_interval


print('lin vs ann', calculate_confidence_interval(error_linreg, error_ann))
print('lin vs base', calculate_confidence_interval(error_linreg, error_base))
print('ann vs base', calculate_confidence_interval(error_ann, error_base))

lin vs ann (-0.20161767686315812, -0.09812133465746076)
lin vs base (-0.5750239506831848, -0.39112728194443824)
ann vs base (-0.40231792092495355, -0.2640943001820507)


## Classification 

### Logistic regression

In [24]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from data import *

np.random.seed(2)
torch.manual_seed(2)

# Define the Logistic Regression model in PyTorch.
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

# Convert the numpy arrays to PyTorch tensors
X_torch = torch.from_numpy(X.astype(np.float32))

# Convert the target to binary labels
y = (y > np.median(y)).astype(int)  # 1 if y is above the median, 0 otherwise
y_torch = torch.from_numpy(y.astype(np.float32)).view(-1, 1)

kf = KFold(n_splits=10, shuffle=True)

# Set lambda value
lambda_val = 10**(-1)

# List of important values
error_values = []
y_true_logreg = np.array([])
y_hat_logreg = np.array([])

for train_idx, test_idx in kf.split(X_torch):
    x_train, x_test = X_torch[train_idx], X_torch[test_idx]
    y_train, y_test = y_torch[train_idx], y_torch[test_idx]

    model = LogisticRegressionModel(input_dim=X_torch.shape[1])
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=lambda_val)

    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        outputs = model(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        y_pred = model(x_test)
        y_pred = (y_pred > 0.5).float()

        # Save information
        y_hat_logreg = np.concatenate((y_hat_logreg,y_pred.numpy().flatten()))
        y_true_logreg = np.concatenate((y_true_logreg,y_test.numpy().flatten()))
        val_error = 1 - torch.mean(y_pred.eq(y_test).float()).item()
        error_values.append(val_error)
        
        
np.mean(error_values)        

0.3029797911643982

### ANN

In [52]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from data import *

np.random.seed(2)
torch.manual_seed(2)

class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_units):
        super(ANNModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_units)
        self.layer2 = nn.Linear(hidden_units, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

# Convert the numpy arrays to PyTorch tensors
X_torch = torch.from_numpy(X.astype(np.float32))

# Convert the target to binary labels
y = (y > np.median(y)).astype(int)  # 1 if y is above the median, 0 otherwise
y_torch = torch.from_numpy(y.astype(np.float32)).view(-1, 1)

kf = KFold(n_splits=10, shuffle=True)

# Set lambda value
hidden_units = 10

# List of important values
error_values = []
y_true_class_ann = np.array([])
y_hat_class_ann = np.array([])

for train_idx, test_idx in kf.split(X_torch):
    x_train, x_test = X_torch[train_idx], X_torch[test_idx]
    y_train, y_test = y_torch[train_idx], y_torch[test_idx]

    model = ANNModel(input_dim=X.shape[1], hidden_units=hidden_units)
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        outputs = model(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        y_pred = model(x_test)
        y_pred = (y_pred > 0.5).float()

        # Save information
        y_true_class_ann = np.concatenate((y_true_class_ann,y_test.numpy().flatten()))
        y_hat_class_ann = np.concatenate((y_hat_class_ann,y_pred.numpy().flatten()))
        
        val_error = 1 - torch.mean(y_pred.eq(y_test).float()).item()
        error_values.append(val_error)
        
        
np.mean(error_values)        

0.43025252521038054

### Baseline

In [48]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from data import *

# Convert the target back to numpy array for sklearn
y_np = y_torch.numpy()

# Create the dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")


# List of important values
error_values = []
y_true_class_base = np.array([])
y_hat_class_base = np.array([])

for train_idx, test_idx in kf.split(X_torch):
    x_train, x_test = X_torch[train_idx], X_torch[test_idx]
    y_train, y_test = y_np[train_idx], y_np[test_idx]

    dummy_clf.fit(x_train, y_train)
    y_pred = dummy_clf.predict(x_test)

    # Save information
    y_true_class_base = np.concatenate((y_true_class_base,y_test.flatten()))
    y_hat_class_base = np.concatenate((y_hat_class_base,y_pred.flatten()))
    val_error = 1 - accuracy_score(y_test, y_pred)
    error_values.append(val_error)
    
np.mean(error_values)

0.5383838383838384

### mcnemar

In [60]:
alpha = 0.05

print("LinReg vs Base")
log_vs_ann = mcnemar(y_true_logreg, y_hat_logreg, y_hat_class_ann, alpha=alpha)
print("theta", log_vs_ann[0], " CI: ", log_vs_ann[1], "p-value", log_vs_ann[2])
print()

print("LogReg vs Base")
log_vs_base = mcnemar(y_true_logreg, y_hat_logreg, y_hat_class_base, alpha=alpha)
print("theta", log_vs_base[0], " CI: ", log_vs_base[1], "p-value", log_vs_base[2])
print()

print("ANN vs Base")
ann_vs_base = mcnemar(y_true_class_ann, y_hat_class_ann, y_hat_class_base, alpha=alpha)
print("theta", ann_vs_base[0], " CI: ", ann_vs_base[1], "p-value", ann_vs_base[2])
print()




LinReg vs Base
Result of McNemars test using alpha= 0.05
Comparison matrix n
[[200. 108.]
 [ 52.  82.]]
Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] =  (0.07174521528095323, 0.18126717991609032)
p-value for two-sided test A and B have same accuracy (exact binomial test): p= 1.1307691767152704e-05
theta 0.12669683257918551  CI:  (0.07174521528095323, 0.18126717991609032) p-value 1.1307691767152704e-05

LogReg vs Base
Result of McNemars test using alpha= 0.05
Comparison matrix n
[[143. 165.]
 [ 59.  75.]]
Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] =  (0.17692836754280727, 0.3017308673170236)
p-value for two-sided test A and B have same accuracy (exact binomial test): p= 8.507217716900644e-13
theta 0.2398190045248869  CI:  (0.17692836754280727, 0.3017308673170236) p-value 8.507217716900644e-13

ANN vs Base
Result of McNemars test using alpha= 0.05
Comparison matrix n
[[147. 105.]
 [ 55. 135.]]
Approximate 1-alpha confidence interval of theta: [t